In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import matplotlib.pyplot as plt
pyo.init_notebook_mode()
from geopy.geocoders import Nominatim

In [ ]:
cd ..

In [3]:
from dotenv import dotenv_values

config = dotenv_values("./.env") 


In [4]:
host = config['HOST']
password = config['PASS']

url = f'postgresql://postgres:{password}@{host}:5432/climate'

In [5]:
engine = create_engine(url, echo=False)

# Goal: Design an interactive climate data visualization 

### Below I examine the temporal fluctuation in yearly mean temperature, recorded by one Irish weather station

In [6]:
with engine.begin() as conn:
    result = conn.execute(text("""
    
    select *
    from yearly_mean_temperature
    where staid = 964
    order by year asc;
    
    """))
                     
data = result.all()
    
ballyshannon_temps = pd.DataFrame(data)

In [7]:
ballyshannon_temps.head(5)

,yearly_temp,year,staid
0,117.5224489795918367,1971,964
1,90.3196721311475410,1972,964
2,96.2958904109589041,1973,964
3,93.0958904109589041,1974,964
4,98.9232876712328767,1975,964


In [8]:
fig = px.line(ballyshannon_temps, x='year', y='yearly_temp', title='Yearly Average Temperature in Ballyshannon, Ireland', markers=True)
fig.show(renderer ='notebook_connected')
fig.write_html('ballyshannon_temps.html')


### Below I examine the temporal fluctuation in yearly mean temperature across Ireland

In [9]:
with engine.begin() as conn:
    result = conn.execute(text("""
    
    select staid, staname
    from stations
    where cn = 'IE';
    
    """))


In [10]:
with engine.begin() as conn:
    result = conn.execute(text("""
    
SELECT stations.staname as staname, stations.latitude, stations.longitude, stations.staid as staid, yearly_mean_temperature.year as year, yearly_mean_temperature.yearly_temp as yearly_temp
FROM yearly_mean_temperature
LEFT JOIN stations 
ON stations.staid = yearly_mean_temperature.staid
WHERE stations.cn = 'IE'
ORDER BY yearly_mean_temperature.year asc
    """))
                     
data = result.all()
ie_data = pd.DataFrame(data)
ie_data

,staname,latitude,longitude,staid,year,yearly_temp
0,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1861,103.0530726256983240
1,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1863,103.6121794871794872
2,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1864,96.2136986301369863
3,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1865,102.6054794520547945
4,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1866,98.5178082191780822
...,...,...,...,...,...,...
2317,MOOREPARK,52.16388888888888888889,-8.26388888888888888889,3552,2023,85.2553191489361702
2318,BALLYHAISE,54.05083333333333333333,-7.31000000000000000000,3553,2023,81.5827814569536424
2319,SHERKIN ISLAND,51.47583333333333333334,-9.42805555555555555556,3554,2023,92.8368794326241135
2320,MULLINGAR,53.53666666666666666666,-7.36166666666666666667,3555,2023,78.2269503546099291


In [11]:
ie_data.isna().value_counts()

staname  latitude  longitude  staid  year   yearly_temp
False    False     False      False  False  False          2322
Name: count, dtype: int64

In [12]:
fig = px.line(ie_data, x='year', y='yearly_temp', color='staname')
fig.show(renderer ='notebook_connected')
fig.write_html('ireland_temps.html')

### How do daily temperatures fluctuate over time? What does that tell us about climate patterns?

In [13]:
with engine.begin() as conn:
    result = conn.execute(text("""
    DROP TABLE IF EXISTS mean_temp_ie;
    
    CREATE TABLE mean_temp_ie
    AS (SELECT stations.staname as staname, 
    stations.staid as staid,
    mean_temperature.tg/10 as temp_celcius,
    mean_temperature.date,
    DATE_PART('day', date):: INT as day, 
    DATE_PART('month', date):: INT as month,
    DATE_PART('year', date):: INT as year
FROM mean_temperature
FULL JOIN stations 
ON mean_temperature.staid = stations.staid
WHERE stations.cn = 'IE');"""))

In [14]:
with engine.begin() as conn:
    result = conn.execute(text("""
    DROP TABLE IF EXISTS min_max_ie;
    CREATE TABLE min_max_ie AS(
    SELECT DISTINCT 
    staname,
    staid, 
    year,
    MIN(temp_celcius) OVER(PARTITION BY year) AS yearly_min,
    MAX(temp_celcius) OVER(PARTITION BY year) AS yearly_max
    FROM mean_temp_ie);"""))


In [15]:
with engine.begin() as conn:
    result = conn.execute(text("""
    SELECT *
    FROM min_max_ie
    ORDER BY year ASC;"""))

data = result.all()
pd.DataFrame(data)

,staname,staid,year,yearly_min,yearly_max
0,GALWAY,441,1861,0,20
1,GALWAY,441,1863,4,19
2,GALWAY,441,1864,0,19
3,GALWAY,441,1865,-2,22
4,GALWAY,441,1866,0,20
...,...,...,...,...,...
2317,SHERKIN ISLAND,3554,2023,-2,16
2318,CASEMENT AERODROME,970,2023,-2,16
2319,ATHENRY,11008,2023,-2,16
2320,BIRR,120,2023,-2,16


In [16]:
with engine.begin() as conn:
    result = conn.execute(text("""
    SELECT *
    FROM mean_temp_ie
    WHERE year = 1973
    ORDER BY date asc;"""))

data = result.all()
ie_1973 = pd.DataFrame(data)

In [17]:
max_73 = ie_1973['temp_celcius'].max()

In [18]:
max_73 = ie_1973['temp_celcius'].max()
min_73 = ie_1973['temp_celcius'].min()

fig = px.scatter(ie_1973, x='date',y='temp_celcius', color='staname')
fig.add_hline(y=max_73, line_width=3, line_dash='dash', line_color='red')
fig.add_hline(y=min_73, line_width=3, line_dash='dash', line_color='blue')
fig.show(renderer ='notebook_connected')
fig.write_html('1973_temps.html')

In [19]:
with engine.begin() as conn:
    result = conn.execute(text("""
    SELECT *
    FROM mean_temp_ie
    WHERE year = 1983
    ORDER BY date asc;"""))

data = result.all()
ie_1983 = pd.DataFrame(data)

In [20]:
max_83 = ie_1983['temp_celcius'].max()
min_83 = ie_1983['temp_celcius'].min()

fig = px.scatter(ie_1983, x='date',y='temp_celcius', color='staname')
fig.add_hline(y=max_83, line_width=3, line_dash='dash', line_color='red')
fig.add_hline(y=min_83, line_width=3, line_dash='dash', line_color='blue')
fig.show(renderer ='notebook_connected')
fig.write_html('1983_temps.html')

In [21]:
with engine.begin() as conn:
    result = conn.execute(text("""
    SELECT *
    FROM mean_temp_ie
    WHERE year = 1993
    ORDER BY date asc;"""))

data = result.all()
ie_1993 = pd.DataFrame(data)

In [22]:
max_93 = ie_1993['temp_celcius'].max()
min_93 = ie_1993['temp_celcius'].min()

fig = px.scatter(ie_1993, x='date',y='temp_celcius', color='staname')
fig.add_hline(y=max_93, line_width=3, line_dash='dash', line_color='red')
fig.add_hline(y=min_93, line_width=3, line_dash='dash', line_color='blue')
fig.show(renderer ='notebook_connected')
fig.write_html('1993_temps.html')

In [23]:
with engine.begin() as conn:
    result = conn.execute(text("""
    SELECT *
    FROM mean_temp_ie
    WHERE year = 2003
    ORDER BY date asc;"""))

data = result.all()
ie_2003 = pd.DataFrame(data)

In [24]:
max_03 = ie_2003['temp_celcius'].max()
min_03 = ie_2003['temp_celcius'].min()


fig = px.scatter(ie_2003, x='date',y='temp_celcius', color='staname')
fig.add_hline(y=max_03, line_width=3, line_dash='dash', line_color='red')
fig.add_hline(y=min_03, line_width=3, line_dash='dash', line_color='blue')
fig.show(renderer ='notebook_connected')
fig.write_html('2003_temps.html')

In [25]:
with engine.begin() as conn:
    result = conn.execute(text("""
    SELECT *
    FROM mean_temp_ie
    WHERE year = 2013
    ORDER BY date asc;"""))

data = result.all()
ie_2013 = pd.DataFrame(data)

In [26]:
max_13 = ie_2013['temp_celcius'].max()
min_13 = ie_2013['temp_celcius'].min()


fig = px.scatter(ie_2013, x='date',y='temp_celcius', color='staname')
fig.add_hline(y=max_13, line_width=3, line_dash='dash', line_color='red')
fig.add_hline(y=min_13, line_width=3, line_dash='dash', line_color='blue')
fig.show(renderer ='notebook_connected')
fig.write_html('2013_temps.html')

In [27]:
with engine.begin() as conn:
    result = conn.execute(text("""
    SELECT *
    FROM mean_temp_ie
    WHERE year = 2022
    ORDER BY date asc;"""))

data = result.all()
ie_2022 = pd.DataFrame(data)

In [28]:
max_22 = ie_2022['temp_celcius'].max()
min_22 = ie_2022['temp_celcius'].min()


fig = px.scatter(ie_2022, x='date',y='temp_celcius', color='staname')
fig.add_hline(y=max_22, line_width=3, line_dash='dash', line_color='red')
fig.add_hline(y=min_22, line_width=3, line_dash='dash', line_color='blue')
fig.show(renderer ='notebook_connected')
fig.write_html('2022_temps.html')

### Below is an interactive map of station locations across the country:

In [29]:
ie_data

,staname,latitude,longitude,staid,year,yearly_temp
0,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1861,103.0530726256983240
1,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1863,103.6121794871794872
2,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1864,96.2136986301369863
3,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1865,102.6054794520547945
4,GALWAY,53.28000000000000000000,-9.02000000000000000000,441,1866,98.5178082191780822
...,...,...,...,...,...,...
2317,MOOREPARK,52.16388888888888888889,-8.26388888888888888889,3552,2023,85.2553191489361702
2318,BALLYHAISE,54.05083333333333333333,-7.31000000000000000000,3553,2023,81.5827814569536424
2319,SHERKIN ISLAND,51.47583333333333333334,-9.42805555555555555556,3554,2023,92.8368794326241135
2320,MULLINGAR,53.53666666666666666666,-7.36166666666666666667,3555,2023,78.2269503546099291


In [30]:
loc_center = Nominatim(user_agent='mymap').geocode('Trinity College Dublin')

print(loc_center.address, loc_center.latitude, loc_center.longitude)

Trinity College Dublin, College Green, Mansion House A Ward 1986, Dublin, County Dublin, Leinster, D02 VR66, Éire / Ireland 53.34366745 -6.254444724511822


In [31]:
df = ie_data
fig = px.scatter_mapbox(
                        df,
                        lat='latitude', 
                        lon='longitude', 
                        zoom=4,
                        center={'lat':  53.34366745, 'lon': -6.254444724511822}, 
                        hover_name='staname', 
                        color='staname',  
                        mapbox_style='open-street-map'
                       )

fig.show(renderer ='notebook_connected')
fig.write_html('ireland_stations.html')